In [1]:
import pandas as pd
from torch.utils.data import DataLoader
from data_files.dataset import CQT_Dataset_test
import torch
from utils.model import Spice_model
import numpy as np
from utils.calibration import Calibrator
from utils.calibration2 import Calibrator_SPICE
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub

import numpy as np
import matplotlib.pyplot as plt
import librosa
from librosa import display as librosadisplay
import pandas as pd

import logging
import math
import statistics
import sys
import random
from tqdm import tqdm

from IPython.display import Audio, Javascript,display, Math, Latex
from scipy.io import wavfile
from scipy import stats

from base64 import b64decode

import music21
from pydub import AudioSegment
from jupyterplot import ProgressPlot
import numpy as np


logger = logging.getLogger()
logger.setLevel(logging.ERROR)

print("tensorflow: %s" % tf.__version__)
#print("librosa: %s" % librosa.__version__)

2023-10-09 23:49:20.692839: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 23:49:24.076262: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


tensorflow: 2.12.0


In [2]:
%env http_proxy=http://proxy:80
%env https_proxy=http://proxy:80

env: http_proxy=http://proxy:80
env: https_proxy=http://proxy:80


# Load Models

In [ ]:

MiniSPICE = Spice_model([1, 64, 128, 256, 512, 512, 512], [512, 512, 512, 256, 128, 64, 1], [True, True, True, True, True, True])
checkpoint = torch.load('./rev_1k_checkpoints/MIR_minispice.ckp', 'cuda' if torch.cuda.is_available() else 'cpu')
MiniSPICE.load_state_dict(checkpoint['state_dict'])

worecon = Spice_model([1, 64, 128, 256, 512, 512, 512], [512, 512, 512, 256, 128, 64, 1], [True, True, True, True, True, True])
checkpoint = torch.load('./rev_1k_checkpoints/wo-recon.ckp', 'cuda' if torch.cuda.is_available() else 'cpu')
worecon.load_state_dict(checkpoint['state_dict'])

SPICE = hub.load("https://tfhub.dev/google/spice/2")

## Calibration

In [ ]:
%%capture
MiniSPICE_cal = Calibrator(MiniSPICE, 1000, 110, 440)
MiniSPICE_PT_OFFSET, MiniSPICE_PT_SLOPE = MiniSPICE_cal.get_values()
MiniSPICE_data, MiniSPICE_A, MiniSPICE_B= MiniSPICE_cal.get_data()
# MiniSPICE_x = np.linspace(0.2, 0.7, 50)
# MiniSPICE_y = MiniSPICE_PT_OFFSET + MiniSPICE_PT_SLOPE*MiniSPICE_x

worecon_cal = Calibrator(worecon, 1000, 110, 440)
worecon_PT_OFFSET, worecon_PT_SLOPE = worecon_cal.get_values()
worecon_data, worecon_A, worecon_B= worecon_cal.get_data()
# worecon_x = np.linspace(0.2, 0.7, 50)
# worecon_y = worecon_PT_OFFSET + worecon_PT_SLOPE*worecon_x

SPICE_cal = Calibrator_SPICE(SPICE, False,1000)
SPICE_PT_OFFSET, SPICE_PT_SLOPE = SPICE_cal.get_values()
SPICE_A, SPICE_B= SPICE_cal.get_data()
# SPICE_x = np.linspace(0.2, 0.7, 50)
# SPICE_y = SPICE_PT_OFFSET + SPICE_PT_SLOPE*SPICE_x

In [ ]:
SPICE_x = np.linspace(0.2, 0.7, 50)
SPICE_y = SPICE_PT_OFFSET + SPICE_PT_SLOPE*SPICE_x
worecon_x = np.linspace(0.35, 0.55, 50)
worecon_y = worecon_PT_OFFSET + worecon_PT_SLOPE*worecon_x
MiniSPICE_x = np.linspace(0.2, 0.35, 50)
MiniSPICE_y = MiniSPICE_PT_OFFSET + MiniSPICE_PT_SLOPE*MiniSPICE_x

# Understanding Calibration Estimation 

## Progressplot (Not Required)

In [ ]:

# pp = ProgressPlot(line_names=["SPICE", "w/o Recon", "Mini-SPICE"],
#                   x_iterator=False, x_label="LHS")
SPICE_pp = ProgressPlot(line_names=["SPICE"],
                   x_iterator=False, x_label="L.H.S")
# worecon_pp = ProgressPlot(line_names=["w/o Recon"],
#                    x_iterator=False, x_label="LHS")
MiniSPICE_pp = ProgressPlot(line_names=["Mini-SPICE"],
                   x_iterator=False, x_label="LHS")
SPICE_dict = {}
worecon_dict = {}
MiniSPICE_dict = {}

for i,j,k,l,m,n in zip(SPICE_B.squeeze(),SPICE_A[:,1],
                       worecon_B.squeeze(),worecon_A[:,1],
                       MiniSPICE_B.squeeze(),MiniSPICE_A[:,1]):
    SPICE_dict[j]=i
    worecon_dict[l]=k
    MiniSPICE_dict[n]=m
    
SPICE_dict_srt = dict(sorted(SPICE_dict.items(), key=lambda item: item[1]))
worecon_dict_srt = dict(sorted(worecon_dict.items(), key=lambda item: item[1]))
MiniSPICE_dict_srt = dict(sorted(MiniSPICE_dict.items(), key=lambda item: item[1]))
for i,j,k in zip(SPICE_dict_srt.items(),worecon_dict_srt.items(),
                      MiniSPICE_dict_srt.items()):
    # SPICE_pp.update(i[1], i[0])
    # worecon_pp.update(j[1], j[0])
    MiniSPICE_pp.update(k[1], k[0])



# SPICE_pp.finalize()
# worecon_pp.finalize()
MiniSPICE_pp.finalize()
display(Math(r'L.H.S = 12 \log _2(\frac{\hat{f}_{0, t}}{f_{\text {base }}})'))


## Scatter

In [ ]:
from matplotlib import pyplot as plt

plt.scatter(SPICE_A[:,1],SPICE_B.squeeze(), c='red',label='SPICE Datapoints')
plt.plot(SPICE_x, SPICE_y, 'lawngreen',linestyle='--',label='SPICE: Estimated Line')
plt.scatter(MiniSPICE_A[:,1],MiniSPICE_B.squeeze(), c='blue',label='Mini-SPICE Datapoints')
plt.plot(MiniSPICE_x, MiniSPICE_y, 'aqua',linestyle='--',label='Mini-SPICE: Estimated Line')
plt.scatter(worecon_A[:,1],worecon_B.squeeze(), c='cyan', label='w/o Recon Datapoints')
plt.plot(worecon_x, worecon_y, 'navy',linestyle='--',label='w/o Recon: Estimated Line')
plt.xlabel('Pitch Head Output/ Enc(x)')
plt.ylabel('Pith Diff from fmin(10Hz) [Semitones]')
plt.legend(bbox_to_anchor=(0.9, 0.45))

display(Math(r'{X_{\text {axis }}} = Enc(input)'))
display(Math(r'{Y_{\text {axis }}} = 12 \log _2(\frac{\hat{f}_{0, t}}{f_{\text {base }}})'))


# Pitch Head vs Frequency

## Setup

In [ ]:
def pick_semitone_freq(low, high):
    semitone_ratio = 2 ** (1/12)
    semitone_freq_list = []
    
    current_freq = low
    while current_freq <= high:
        semitone_freq_list.append(current_freq)
        current_freq *= semitone_ratio
    
    return random.choice(semitone_freq_list)

# pick_semitone_freq(110, 440)

def generate_harmonic_wave(fundamental_freq, sampling_rate=16000, N=11, H=512):
    amplitude_f0 = np.random.normal(0, 1)
    t = np.linspace(0, (N*H) / sampling_rate, (N*H), endpoint=False)
    fundamental_signal = amplitude_f0*np.sin(2 * np.pi * fundamental_freq * t)
    harmonic_signal = fundamental_signal.copy()
    

    for i in range(2, 4):  
        frequency = fundamental_freq * i
        random_phase = random.uniform(0, 2 * math.pi)
        harmonic_signal += random.uniform(0, 1) * np.sin(2 * np.pi * frequency * t+random_phase)

    return harmonic_signal

class Generator():
    def __init__(self,M,low=110,high=440):
        self.M = M
        self.low = low
        self.high = high
        
    def generate(self):
        samples = []
        labels = []
        for i in range(self.M):
            f0 = pick_semitone_freq(self.low,self.high)
            wave = generate_harmonic_wave(f0)
            samples.append(wave)
            labels.append(f0)
        
        return np.array(samples), np.array(labels)

In [ ]:
genrator = Generator(1000,20,2000)
audio, labels = genrator.generate()

In [ ]:
print("******* Generating CQT Samples *******\n")
Cqtt = np.zeros((1, 190))
F0 = np.zeros(1)
for s, f in zip(audio, labels):
    C = np.abs(librosa.cqt(s, sr=16000, hop_length=512, 
                fmin= librosa.note_to_hz('C1'),
                n_bins=190, bins_per_octave=24))
    Cqtt = np.vstack((Cqtt, C.T))
    f0 = np.repeat(f, C.shape[1])
    F0 = np.concatenate((F0, f0))
Cqtt = Cqtt[1:, :]
F0 = F0[1:].reshape(-1, 1)
Cqt_data = np.hstack((Cqtt, F0))
print("******* Generating CQT Samples Completed *******\n")

In [ ]:
### FOR Mini-SPICE
MiniSPICE_pitch_h = np.array([])
MiniSPICE_labels= np.array([])
for row in tqdm(Cqt_data):
    pitch_h1,conf_h1,x_hat1 = MiniSPICE(torch.from_numpy(row[0:128].reshape(1,128)).float())
    MiniSPICE_pitch_h = np.append(MiniSPICE_pitch_h, np.array(pitch_h1.detach().numpy()))
    MiniSPICE_labels = np.append(MiniSPICE_labels, np.array(row[-1]))

In [ ]:
def output2hz(pitch_output):
  # Constants taken from https://tfhub.dev/google/spice/2
  PT_OFFSET = 25.58
  PT_SLOPE = 63.07
  FMIN = 10.0;
  BINS_PER_OCTAVE = 12.0;
  cqt_bin = pitch_output * PT_SLOPE + PT_OFFSET;
  return FMIN * 2.0 ** (1.0 * cqt_bin / BINS_PER_OCTAVE)

In [ ]:
### For SPICE

SPICE_pitch_h = np.array([])
SPICE_labels= np.array([])
for s,f in zip(audio, labels):
    # print(s)
    model_output = SPICE.signatures["serving_default"](tf.constant(list(s), tf.float32))["pitch"]
    pitch_outputs = [ float(x) for x in model_output]
    SPICE_pitch_h = np.append(SPICE_pitch_h, np.array(pitch_outputs))
    label = [f] * 12
    SPICE_labels = np.append(SPICE_labels, np.array(label))

## Plots

In [ ]:

MiniSPICE_x = MiniSPICE_labels
MiniSPICE_y = MiniSPICE_pitch_h
SPICE_x = SPICE_labels
SPICE_y = SPICE_pitch_h
SPICE_bins = np.linspace(min(SPICE_x), max(SPICE_x), num=20) 
SPICE_bin_means, SPICE_bin_edges, SPICE_binnumber = stats.binned_statistic(SPICE_x, SPICE_y, 
                                                                           statistic='mean', bins=SPICE_bins)

MiniSPICE_bins = np.linspace(min(MiniSPICE_x), max(MiniSPICE_x), num=20) 
MiniSPICE_bin_means, MiniSPICE_bin_edges, MiniSPICE_binnumber = stats.binned_statistic(MiniSPICE_x, MiniSPICE_y, 
                                                                           statistic='mean', bins=MiniSPICE_bins)
plt.plot(SPICE_bin_edges[:-1], SPICE_bin_means,color='g', label='SPICE Outputs')
plt.plot(MiniSPICE_bin_edges[:-1], MiniSPICE_bin_means, color='b', label='Mini-SPICE Outputs')
# fig, ax = plt.subplots()
plt.axvline(x=46.259328, color='r', linestyle='--', label='MDB fmin')
# ax.axvline(x=46.259328, color='r', linestyle='--', label='Vertical Line at x=0.5')
# plt.text(46.259328,0,'blah',rotation=90)
plt.axvline(x=66.9456331525636256, color='c', linestyle='--', label='MIR1k fmin')
plt.axvline(x=666.664939769901, color='m', linestyle='--', label='MIR1k fmax')
plt.axvline(x=1210.180328, color='y', linestyle='--', label='MDB fmax')
plt.xlabel('Frequency in Hz')
plt.ylabel('Binwise Mean Pitch Head Output')
# plt.title('Frequency Vs. SPICE Pitch Head Output')
plt.legend(bbox_to_anchor=(1.0, 0.45))

plt.show()